# Taller 4: Redes Neuronales Recurrentes

**Jessenia Piza, Laura Alejandra Salazar & Paula Lorena López**

Este taller consiste en actualizar el taller anterior (3), es decir, usar redes neuronales recurrentes, en vez de redes neuronales multicapa (MLP, multilayer perceptron).

## Ejercicio 1. 
Use el dataset de `progressive-tweet-sentiment.csv` (el del taller 1 y taller 3), para realizar una clasificación de las 4 clases en que pertenece cada tweet, usando redes neuronales recurrentes. Es decir, que esta vez, no va a usar sólo capas MLP, sino una o varias capas recurrentes, seguida de una capa lineal (full connected, o densa). Pruebe diferentes configuraciones (use recurrente básica, LSTM, GRU y bidireccionalidad). 

Debe utilizar pytorch. Grafique el loss y el accuracy, tanto para el entrenamiento como para la validación. Escoja el mejor modelo probando con un buen número de epochs (use el optimizador de Adam con learning rate por defecto de 0.001).

Recuerde que en este ejercicio no va a usar word2vec, sino una capa de embeddings.

A continuación muestro como leer el conjunto de datos, crear un `Dataset`de pytorch y dividir en un training y validation set (si se quiere un test se puede repetir el proceso)

In [1]:
import torch
import pandas as pd
import numpy as np
import re
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import torch.nn as nn
import nltk
from nltk.tokenize import TweetTokenizer
from torchtext.vocab import vocab
from collections import Counter, OrderedDict
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [2]:
import torch.nn.functional as F
import torch.optim as optim
from unicodedata import normalize

Se tiene una función que lee la base de  datos con la que se trabajará.
Se guarda y se divide en `train_dataset` y `valid_dataset`.


In [3]:
class TextData(Dataset):
    '''
    Dataset basico para leer los datos de tweets
    '''
    def __init__(self, filename):
        super(TextData, self).__init__()
        df = pd.read_csv(filename,encoding='latin-1')
        self.df = df[["target", "tweet"]]
        
    def __getitem__(self, index):
        return self.df.iloc[index,0], self.df.iloc[index,1]
    
    def __len__(self):
        return len(self.df)

In [4]:
ds = TextData("progressive-tweet-sentiment.csv")
train_dataset, valid_dataset = random_split(ds,
 [int(len(ds)*0.7),len(ds) - int(len(ds)*0.7)], torch.manual_seed(42))

Una vez hecho esto, se limpían todos los tweets con los que se trabajarán, de manera que queda más fácil el análisis de este.

In [5]:
import re
from collections import Counter, OrderedDict

token_counts = Counter()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized


for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens)
 
    
print('Vocab-size:', len(token_counts))

Vocab-size: 3785


In [6]:
sorted_by_freq_tuples = sorted(token_counts.items(),
                               key=lambda x: x[1], reverse=True)
ordered_dict = OrderedDict(sorted_by_freq_tuples)

vocab = vocab(ordered_dict)

vocab.insert_token("<pad>", 0)
vocab.insert_token("<unk>", 1)
vocab.set_default_index(1)


De esta manera, ya tenemos un vocaulario creado.


Se definen las funciones para la transformación y de esta manera, se aplican para la función de codificación y transformación. 

Esto lo que permite es retornar el texto de su respectivo batch con su padding y lables.

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
lbl = {'Feminist Movement':0, 'Hillary Clinton':1,
       'Legalization of Abortion':2, 'Atheism': 3}
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: lbl[x]

In [8]:
def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), 
                                      dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)
    padded_text_list = nn.utils.rnn.pad_sequence(
        text_list, batch_first=True)
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)

Se crea el Dataloader de train y de valid. 
Se utiliza un batch pequeño dado que la longitud de los tweets no es tan larga.

In [9]:
batch_size = 40
train_dl = DataLoader(train_dataset, batch_size=batch_size,
                      shuffle=True, collate_fn=collate_batch)
valid_dl = DataLoader(valid_dataset, batch_size=batch_size,
                      shuffle=False, collate_fn=collate_batch)


Se crea la clase de la Red Neuronal Recurrente.
Se tabaja con una RNN bidereccional. Esto quiere decir que la capa recurrente es bidireccional. Así que pasa por vaarias capas entre ella están _embedding, LSTM, Linear_.

El tamaño del batch es el mismo que para la construcción del Dataloader.

In [10]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 
                                      embed_dim, 
                                      padding_idx=0) 
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(),
                                                enforce_sorted=False,
                                                batch_first=True)
        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [11]:
vocab_size = len(vocab)
embed_dim = 25
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)

Se crean las funciones de entrenamiento y evaluación del modelo.

In [12]:
def train(dataloader):
    model.train()
    total_acc, total_loss = 0, 0
    for text_batch, label_batch, lengths in dataloader:
        optimizer.zero_grad()
        pred = model(text_batch, lengths)
        loss = loss_fn(pred, label_batch.type(torch.LongTensor))
        loss.backward()
        optimizer.step()
        var = (torch.argmax(pred, dim=1)).float()
        total_acc += (var == label_batch).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)
 
def evaluate(dataloader):
    model.eval()
    total_acc, total_loss = 0, 0
    with torch.no_grad():
        for text_batch, label_batch, lengths in dataloader:
            pred = model(text_batch, lengths)
            loss = loss_fn(pred, label_batch.type(torch.LongTensor))        
            var = (torch.argmax(pred, dim=1)).float()
            total_acc += (var == label_batch).float().sum().item()
            total_loss += loss.item()*label_batch.size(0)
    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)

Se utiliza `CrossEntropyLoss` como función de pérdida y el optimizador de `Adam` para evaluar el modelo. 
Además se utilizan 10 épocas para el entrenamiento.

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10 

torch.manual_seed(1)
list_loss_train = []
list_loss_valid = []
list_acc_train = []
list_acc_valid = []

for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    list_loss_train.append(loss_train)
    list_loss_valid.append(loss_valid)
    list_acc_train.append(acc_train)
    list_acc_valid.append(acc_valid)
    print(f'Epoch {epoch} loss: {loss_train:.4f} val_loss: {loss_valid:.4f} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')
 
 

Epoch 0 loss: 1.3845 val_loss: 1.3830 accuracy: 0.2713 val_accuracy: 0.2701
Epoch 1 loss: 1.3805 val_loss: 1.3790 accuracy: 0.2935 val_accuracy: 0.2845
Epoch 2 loss: 1.3722 val_loss: 1.3650 accuracy: 0.3255 val_accuracy: 0.3046
Epoch 3 loss: 1.3455 val_loss: 1.3411 accuracy: 0.4192 val_accuracy: 0.3362
Epoch 4 loss: 1.2912 val_loss: 1.3089 accuracy: 0.4439 val_accuracy: 0.3851
Epoch 5 loss: 1.2272 val_loss: 1.2803 accuracy: 0.5216 val_accuracy: 0.4138
Epoch 6 loss: 1.1781 val_loss: 1.2512 accuracy: 0.5832 val_accuracy: 0.4483
Epoch 7 loss: 1.1239 val_loss: 1.2422 accuracy: 0.6301 val_accuracy: 0.4626
Epoch 8 loss: 1.0934 val_loss: 1.2443 accuracy: 0.6523 val_accuracy: 0.4598
Epoch 9 loss: 1.0535 val_loss: 1.2199 accuracy: 0.6954 val_accuracy: 0.4885


## Ejercicio 2.
Ahora proceda a actualizar el ejercicio 5 del taller 3, pero usando RNNs. Es decir, debe solucionar un problema de Name Entity Recognition (NER), con un dataset pequeño creado por ustedes (con varias entidades), pero usando redes recurrentes (RNN básica, LSTM, GRU). Use GRU o LSTM.

Este ejercicio es en realidad más simple que el del anterior taller, ya que no hay necesidad de organizar las sentencias centradas en cada palabra, sino que se toma directamente cada frase (o secuencia). 

Lo importante ahora, es utilizar todos los estados de salida de la red recurrente y llevarlos a una capa lineal de clasificación. 

[Este tutorial](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html) le puede ser muy útil para desarrollar este punto. 

Utilizaremos el texto del anterior taller para realizar este punto.

In [14]:
corpus = [
          "Nosotros siempre venimos a París",
          "El profesor es de Australia",
          "Yo vivo en Bogotá",
          "Él viene de Taiwán",
          "La capital de Turquía es Ankara"
         ]

Recordemos que la función de preprocesamiento que usaremos para generar nuestros ejemplos de entrenamiento.
Es decir, ponemos las letras en minúsculas, quitamos tildes y luego tokenizamos las palabras.

In [15]:
def quitartildes(s):
    # -> NFD y eliminar diacríticos
    s = re.sub(
            r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
            normalize( "NFD", s), 0, re.I
        )

    # -> NFC
    return normalize( 'NFC', s)

def preprocess_sentence(sentence):
  return quitartildes(sentence).lower().split()

# Crea nuestro conjunto de entrenamiento
train_sentences = [preprocess_sentence(sent) for sent in corpus]
train_sentences

[['nosotros', 'siempre', 'venimos', 'a', 'paris'],
 ['el', 'profesor', 'es', 'de', 'australia'],
 ['yo', 'vivo', 'en', 'bogota'],
 ['el', 'viene', 'de', 'taiwan'],
 ['la', 'capital', 'de', 'turquia', 'es', 'ankara']]

A diferencia del taller anterior, agregamos un nuevo target para los labels. 

Tenemos `locations` y `verbs`, en donde si la palabra corresponde a la primera generará `1` y `2` si es un verbo. De otra manera, generará `0`.

In [16]:
locations = set(["australia", "ankara", "paris", "bogota", "taiwan", "turquia"])
verbs = set(['venimos', 'es', 'vivo', 'viene', 'es'])

In [17]:
tag_to_ix = {'other': 0, 'location': 1, 'verb':2}

In [18]:
train_labels = []

for i in train_sentences:
  part = []
  for j in i:
    if j in locations:
      part.append('location')
    elif j in verbs:
      part.append('verb')
    else:
      part.append('other')
  train_labels.append(part)

In [19]:
training_data = [(train_sentences[i],train_labels[i]) for i in range(len(corpus))]

In [20]:
training_data

[(['nosotros', 'siempre', 'venimos', 'a', 'paris'],
  ['other', 'other', 'verb', 'other', 'location']),
 (['el', 'profesor', 'es', 'de', 'australia'],
  ['other', 'other', 'verb', 'other', 'location']),
 (['yo', 'vivo', 'en', 'bogota'], ['other', 'verb', 'other', 'location']),
 (['el', 'viene', 'de', 'taiwan'], ['other', 'verb', 'other', 'location']),
 (['la', 'capital', 'de', 'turquia', 'es', 'ankara'],
  ['other', 'other', 'other', 'location', 'verb', 'location'])]

In [21]:
vocabulary = set(w for s in train_sentences for w in s)
vocabulary.add("<unk>")
vocabulary.add("<pad>")

In [22]:
ix_to_word = sorted(list(vocabulary))

In [23]:
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index

Creamos la red neuronal.

In [24]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] if w in to_ix else to_ix["<unk>"] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [25]:
class LSTMTagger(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Se crea el  nuevo modelo con la función de pérdida de `NLLoss`(Negative Log Likelihood) y nuevamente el optimizador `Adam`.

In [26]:
EMBEDDING_DIM = 20
HIDDEN_DIM = 10
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)


In [27]:
for epoch in range(100):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
    print(prepare_sequence(training_data[0][1],tag_to_ix))
    print(torch.argmax(tag_scores, dim=1))

tensor([[-0.0371, -5.1502, -3.4870],
        [-0.0262, -7.0279, -3.6912],
        [-3.8552, -4.4633, -0.0332],
        [-0.0118, -4.6812, -6.0058],
        [-5.0651, -0.0182, -4.4445]])
tensor([0, 0, 2, 0, 1])
tensor([0, 0, 2, 0, 1])


### Nota: puede usar las herramientas de pytorch para crear el vocabulario, y las herramientas de relleno (pad)